In [ ]:
!pip install selenium

In [5]:
from selenium import webdriver
import pandas as pd
driver = webdriver.Chrome('C:/Windows/chromedriver_win32/chromedriver.exe')

In [6]:
df = pd.DataFrame(columns=['username', 'date', 'comments'])
for j in range(250):
  driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p'+str(j+1))
  for i in range(20):
    df.loc[i+j*20, 'username'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[2]/div[1]/span[1]/a[2]')[0].text
    df.loc[i+j*20, 'date'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[2]/div[2]/span/a/time')[0].text
    df.loc[i+j*20, 'comments'] = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div[3]/div[5]/div[1]/ul/li['+ str(i+1) +']/div/div[3]/div/div[1]')[0].text

In [8]:
#Convert to csv file
import copy
comments_clean = copy.deepcopy(df)
comments_clean['comments'] = comments_clean['comments'].apply(lambda x :x.replace("\n"," "))
comments_clean.to_csv('comments.csv', header=True, sep=',')

In [1]:
#Remove punctuation and convert to lower case
import string
import nltk
import pandas as pd
comments_clean = pd.read_csv("comments.csv")
comments_clean['comments']=comments_clean['comments'].astype(str)
comments_clean = comments_clean.dropna()
comments_clean = comments_clean.drop(columns = 'Unnamed: 0')
comments_clean = comments_clean.reset_index().drop(columns='index')
comments_clean['comments'] = comments_clean['comments'].apply(lambda x :x.translate(str.maketrans('', '', string.punctuation)))
comments_clean['comments'] = comments_clean['comments'].apply(lambda x :x.lower())

In [2]:
#Import models to brand conversion file 
models = pd.read_csv("models.csv", header = None, names = ['brand','model'], encoding='windows-1252')

In [3]:
#Replace models by brands
def model_to_brand(s):
    for i in models.index.values:
        s = s.replace(models["model"][i].lower(),models["brand"][i].lower())
    return s
comments_clean['comments_model_replace'] = comments_clean['comments'].apply(model_to_brand)

In [4]:
#Tokenize and remove stop words
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
stop_words = set(stopwords.words('english')) 
comments_clean['comments_model_replace'] = comments_clean['comments_model_replace'].apply(word_tokenize).apply(set).apply(list)

def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
comments_clean['comments_model_replace'] =  comments_clean['comments_model_replace'].apply(remove_stopwords)


In [5]:
#Creating a combined list of all words
all_words = []
for i in range(len(comments_clean)):
    all_words+=comments_clean['comments_model_replace'][i]
from nltk import FreqDist
word_freq = nltk.FreqDist(all_words)

In [6]:
#Creating a list of all brand names
model_names = models['brand'].unique()

In [7]:
#Filtering counts for brands
brand_counts = []
for key,item in word_freq.most_common(1000):
    if key in model_names:
        temp = (key,item)
        brand_counts.append(temp)
        

In [8]:
#Top 10 brands
brand_counts[:10]

[('bmw', 1401),
 ('acura', 529),
 ('audi', 439),
 ('infiniti', 415),
 ('honda', 403),
 ('toyata', 209),
 ('nissan', 191),
 ('volvo', 174),
 ('ford', 143),
 ('mercedes', 143)]